In [ ]:
import pandas
import matplotlib
import matplotlib.pyplot
import mpl_toolkits.axes_grid1
import seaborn

import luminex
import luminex.functions

In [ ]:
version = luminex.__version__
version

In [ ]:
def create_fig_ax(width, height, units='cm'):
    
    if units=='cm':
        width = width*0.393701
        height = height*0.393701
    
    fig = matplotlib.pyplot.figure(figsize=(width+3, height+3))

    rect = (0,0,1,1)
    
    horizontal = [
        mpl_toolkits.axes_grid1.Size.Fixed(1),
        mpl_toolkits.axes_grid1.Size.Fixed(width),
        mpl_toolkits.axes_grid1.Size.Fixed(2),
    ]
    vertical = [
        mpl_toolkits.axes_grid1.Size.Fixed(1),
        mpl_toolkits.axes_grid1.Size.Fixed(height),
        mpl_toolkits.axes_grid1.Size.Fixed(2)
    ]
    divider = mpl_toolkits.axes_grid1.Divider(fig, rect, horizontal, vertical, aspect=False)
    
    ax = fig.add_axes(rect)
    ax.set_axes_locator(divider.new_locator(nx=1, ny=1))

    return fig, ax

In [ ]:
pandas.set_option('display.max_columns', 500)

colourmap = {
    'unstimCD14-':'#aec7e8',
    'unstimCD14+':'#1f77b4',
    'cd3CD14-':'#ffbb78',
    'cd3CD14+':'#ff7f0e',
    'bugsCD14-':'#98df8a',
    'bugsCD14+':'#2ca02c'
}

matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
rawdata = pandas.read_excel(
    '/Users/laurapallett/data/laura_pallett/luminex/03/modified_data_45plex_cd14_induced_002.xlsx'
)

# Filter and create feature labels
rawdata = rawdata[rawdata['condition'].isin(['unstim','cd3','bugs'])]
rawdata = rawdata[rawdata['cell_type'].isin(['CD14+','CD14-'])]
rawdata = rawdata[~rawdata['sample_id'].isin(['MOD82','SLG82','JHW88'])]
rawdata['sample_type'] = rawdata['condition'] + rawdata['cell_type']
rawdata.reset_index(drop=True, inplace=True)

sample_columns = ['sample_id', 'cell_type', 'condition', 'sample_type']
feature_columns = [col for col in rawdata.columns if col not in sample_columns]

rawdata.head()

In [ ]:
selected_features = luminex.functions.select_features(
    rawdata,
    sample_columns,
    feature_columns
)
to_drop = ['CD 40Lig','Eotaxin','FGF basic','flt-3 Lig','PDGF-AA','PD-L1','IFN-alpha']
selected_features = [name for name in selected_features if name not in to_drop]

data = rawdata[sample_columns+selected_features]
averaged_data = luminex.functions.fill_with_averages(data)
trans_data = luminex.functions.transform(averaged_data)

## Circles & Barchart unstim CD14+/-

In [ ]:
sample_types = ['unstimCD14-','unstimCD14+']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.my_barplot(data, colourmap, sample_types, ax, order=sample_types)
matplotlib.pyplot.savefig('figures/unstim_barplot_laura_{}.eps'.format(version))

In [ ]:
sample_types = ['unstimCD14-','unstimCD14+']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.plot_expression_circles(
    df=data,
    columns=selected_features,
    labels=sample_types,
    ax=ax,
    colourmap=colourmap
)
matplotlib.pyplot.savefig('figures/unstim_circles_laura_{}.eps'.format(version))

## Circles & Barchart unstim CD14+/- and CD3 CD14+/-

In [ ]:
sample_types = ['unstimCD14-','unstimCD14+','cd3CD14-','cd3CD14+']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.my_barplot(data, colourmap, sample_types, ax, order=sample_types)
matplotlib.pyplot.savefig('figures/unstim_cd3_barplot_laura_{}.eps'.format(version))

In [ ]:
sample_types = ['unstimCD14+','unstimCD14-','cd3CD14+','cd3CD14-']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.plot_expression_circles(
    df=data,
    columns=selected_features,
    labels=sample_types,
    ax=ax,
    colourmap=colourmap
)
matplotlib.pyplot.show()
matplotlib.pyplot.savefig('figures/unstim_cd3_circles_laura_{}.eps'.format(version))

## Heatmap Showing unstim and CD3 CD14+/-

In [ ]:
sample_types = ['unstimCD14-','cd3CD14-','unstimCD14+','cd3CD14+']

g = luminex.functions.plot_cluster_map(trans_data, sample_types, colourmap)
g.savefig('figures/unstim_cd3_heatmap_laura_{}.eps'.format(version))

## Heatmap Showing unstim, bugs CD3 CD14+/-

In [ ]:
sample_types = ['unstimCD14-','cd3CD14-','bugsCD14-','unstimCD14+','cd3CD14+','bugsCD14+']

g = luminex.functions.plot_cluster_map(trans_data, sample_types, colourmap)
g.savefig('figures/unstim_bugs_cd3_heatmap_laura_{}.eps'.format(version))

## Heatmap showing CD14- unstim, bugs, CD3

In [ ]:
sample_types = ['unstimCD14-','bugsCD14-','cd3CD14-']

g = luminex.functions.plot_cluster_map(trans_data, sample_types, colourmap)
g.savefig('figures/unstim_bugs_cd3_cd14neg_heatmap_laura_{}.eps'.format(version))

In [ ]:
sample_types = ['unstimCD14+','bugsCD14+','cd3CD14+']

g = luminex.functions.plot_cluster_map(trans_data, sample_types, colourmap)
g.savefig('figures/unstim_bugs_cd3_cd14pos_heatmap_laura_{}.eps'.format(version))

## UMAP

In [ ]:
fig, ax = create_fig_ax(5.08, 5.08)
luminex.functions.plot_umap(trans_data, ax, colourmap)
matplotlib.pyplot.savefig('figures/umap_laura_{}.eps'.format(version))

In [ ]:
cd14neg_trans_data = trans_data[trans_data['cell_type']=='CD14-'].copy()

fig, ax = create_fig_ax(5.08, 5.08)
luminex.functions.plot_umap(cd14neg_trans_data, ax, colourmap)
matplotlib.pyplot.savefig('figures/umap_cd14neg_laura_{}.eps'.format(version))

In [ ]:
cd14pos_trans_data = trans_data[trans_data['cell_type']=='CD14+'].copy()

fig, ax = create_fig_ax(5.08, 5.08)
luminex.functions.plot_umap(cd14pos_trans_data, ax, colourmap)
matplotlib.pyplot.savefig('figures/umap_cd14pos_laura_{}.eps'.format(version))

## Circles bugs CD14- vs CD14+

In [ ]:
sample_types = ['bugsCD14-','bugsCD14+']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.plot_expression_circles(
    df=data,
    columns=selected_features,
    labels=sample_types,
    ax=ax,
    colourmap=colourmap
)
matplotlib.pyplot.savefig('figures/bugs_circles_laura_{}.eps'.format(version))

## Circles cd3 CD14- vs CD14+

In [ ]:
sample_types = ['cd3CD14-','cd3CD14+']

fig, ax = create_fig_ax(17, 5.08)
luminex.functions.plot_expression_circles(
    df=data,
    columns=selected_features,
    labels=sample_types,
    ax=ax,
    colourmap=colourmap
)
matplotlib.pyplot.savefig('figures/cd3_circles_laura_{}.eps'.format(version))